In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import nltk
from nltk.corpus import stopwords
import re
import pickle
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
%matplotlib inline
plt.style.use('ggplot')

In [95]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')          
stopWords = list(stopwords.words('english'))   

def is_stock_up(row):
    return True if row[1] < row[4] else False


def tokenize_paragraph(paragraph):
    raw_sentences = tokenizer.tokenize(paragraph)
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(sentence_to_wordlist(raw_sentence.lower()))
    return sentences
            
        
def remove_stopwords(sentences):
    sent_remove_stopwords = []
    for sent in sentences:
        append_sent = []
        for word in sent:
            if word not in stopWords:
                append_sent.append(word)
        sent_remove_stopwords.append(append_sent)
    return sent_remove_stopwords
        
            
def clean_text(paragraph):
    paragraph = paragraph.replace(u'\xa0', u' ')
    sentences = tokenize_paragraph(paragraph)
    sent_remove_stopwords = remove_stopwords(sentences)
    words = []
    for sent in sent_remove_stopwords:
        for word in sent:
            words.append(word)
    return words

#check gensim
def get_paragraph_vectors(tokens): 
    num_tokens = len(tokens)
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokens)]
    global doc2vec
    doc2vec = pickle.load(open('doc2vec.sav', 'rb'))
    return [doc2vec.dv[i] for i in range(num_tokens)]

#check gensim
def get_infered_vectors(tokens):
    return model.infer_vector(tokens)
       
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

def reduce_dimensions(vectors):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # reduce using t-SNE
    print('transforming to vectors...')
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=42)
    print('applying tsne...')
    vectors = tsne.fit_transform(vectors)
    print('appending to list...')
    X = [v[0] for v in vectors]
    y = [v[1] for v in vectors]
    return X, y

def get_numpy_array(pandas_series, num_features=300):
    zeroes = np.empty((len(pandas_series),num_features))
    i = 0
    for vectors in pandas_series:
        zeroes[i] = np.array(vectors)
        i+=1
    return zeroes

def get_autoencoder_result(numpy_array):
    encoded_text = encoder.predict(numpy_array)
    return decoder.predict(encoded_text)

def get_vector_difference(initial_vector, decoded_vector):
    return np.absolute(initial_vector.sum() - decoded_vector.sum())

#check gensim
def get_word_vectors(model):
    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.key_to_index:
        vectors.append(model.wv[word])
        labels.append(word)
    return vectors, labels

def get_date(dt_series,premarket_movement=True):
    years = dt_series.dt.year
    months = dt_series.dt.month
    days = dt_series.dt.day
    dates = []
    for year, month, day in zip(years,months,days):
        if premarket_movement:
            date_string = '{year}-{month}-{day}'.format(day=day+1,month=month,year=year)
        else:
            date_string = '{year}-{month}-{day}'.format(day=day,month=month,year=year)
        dates.append(date_string)
    return dates

def get_day_of_week(dt_series,shift_days=False):
    
    day_of_week = list(dt_series.dt.day_name())
    if shift_days:
        days_mapping={
            'Sunday':'Monday',
            'Monday':'Tuesday',
            'Tuesday':'Wednesday',
            'Wednesday':'Thursday',
            'Thursday':'Friday'
        }
        for i in range(len(day_of_week)):
            real_day = days_mapping[day_of_week[i]]
            day_of_week[i] = days_mapping[day_of_week[i]]
    
    return day_of_week
        
def rearrange_date_string(series):
    dates = series.split('-')
    return dates[2] + '-' + dates[1] + '-' + dates[0]
    
    

In [90]:
df = pd.read_csv('wsb_sentiment.csv')
df = df.drop('Unnamed: 0',axis=1)
df.head()

body  \
0   Did you honestly think that was done on purpose?   
1  Yeah right. Man needed the hard truth &amp; I’...   
2  Jokes on her your saving account is already empty   
3  Have you seen him? Lame white hipster dude who...   
4  This is our time bro. Crsr and psth can wait a...   

                 created_utc  is_submitter  \
0  2020-12-31 19:30:47-05:00         False   
1  2020-12-31 19:30:50-05:00         False   
2  2020-12-31 19:31:59-05:00         False   
3  2020-12-31 19:32:05-05:00         False   
4  2020-12-31 20:16:57-05:00         False   

                                              tokens  length  \
0           ['honestly', 'think', 'done', 'purpose']       4   
1  ['yeah', 'right', 'man', 'needed', 'hard', 'tr...      10   
2  ['jokes', 'saving', 'account', 'already', 'emp...       5   
3  ['seen', 'lame', 'white', 'hipster', 'dude', '...      19   
4  ['time', 'bro', 'crsr', 'psth', 'wait', 'coupl...      13   

                                             vectors  labels  sentiment  \
0  [-0.06739178  0.03808049 -0.12307083  0.043420...       0   0.600000   
1  [ 0.33995894  0.00871463  0.35547084  0.073937...       0  -0.002976   
2  [ 0.32191882  0.05790293 -0.2426322   0.011836...       0  -0.100000   
3  [-0.21993735  0.02008334 -0.11642405 -0.435189...       0  -0.008333   
4  [ 0.20235     0.04972394 -0.4092404  -0.436758...       0   0.000000   

   subjectivity date_strings day_of_week  
0       0.90000   31-12-2020    Thursday  
1       0.53869   31-12-2020    Thursday  
2       0.50000   31-12-2020    Thursday  
3       0.52500   31-12-2020    Thursday  
4       0.00000   31-12-2020    Thursday

In [94]:
df['date_strings'] = df['date_strings'].apply(rearrange_date_string)
df

body  \
0       Did you honestly think that was done on purpose?   
1      Yeah right. Man needed the hard truth &amp; I’...   
2      Jokes on her your saving account is already empty   
3      Have you seen him? Lame white hipster dude who...   
4      This is our time bro. Crsr and psth can wait a...   
...                                                  ...   
51790       See that thanksgiving spike? Yea, that's us.   
51791  Can you explain more? I'm interested but I don...   
51792                            Ocgn more pump and dump   
51793  Pltr sells shares to retards. It’s their busin...   
51794  If an infrastructure bill is passed then Ameri...   

                     created_utc  is_submitter  \
0      2020-12-31 19:30:47-05:00         False   
1      2020-12-31 19:30:50-05:00         False   
2      2020-12-31 19:31:59-05:00         False   
3      2020-12-31 19:32:05-05:00         False   
4      2020-12-31 20:16:57-05:00         False   
...                          ...           ...   
51790  2021-02-16 08:59:37-05:00         False   
51791  2021-02-16 08:59:54-05:00         False   
51792  2021-02-16 09:00:11-05:00         False   
51793  2021-02-16 09:01:17-05:00         False   
51794  2021-02-16 09:01:18-05:00         False   

                                                  tokens  length  \
0               ['honestly', 'think', 'done', 'purpose']       4   
1      ['yeah', 'right', 'man', 'needed', 'hard', 'tr...      10   
2      ['jokes', 'saving', 'account', 'already', 'emp...       5   
3      ['seen', 'lame', 'white', 'hipster', 'dude', '...      19   
4      ['time', 'bro', 'crsr', 'psth', 'wait', 'coupl...      13   
...                                                  ...     ...   
51790      ['see', 'thanksgiving', 'spike', 'yea', 'us']       5   
51791         ['explain', 'interested', 'know', 'terms']       4   
51792                           ['ocgn', 'pump', 'dump']       3   
51793  ['pltr', 'sells', 'shares', 'retards', 'busine...       6   
51794  ['infrastructure', 'bill', 'passed', 'america'...      49   

                                                 vectors  labels  sentiment  \
0      [-0.06739178  0.03808049 -0.12307083  0.043420...       0   0.600000   
1      [ 0.33995894  0.00871463  0.35547084  0.073937...       0  -0.002976   
2      [ 0.32191882  0.05790293 -0.2426322   0.011836...       0  -0.100000   
3      [-0.21993735  0.02008334 -0.11642405 -0.435189...       0  -0.008333   
4      [ 0.20235     0.04972394 -0.4092404  -0.436758...       0   0.000000   
...                                                  ...     ...        ...   
51790  [-0.21178158 -0.18199645  0.29820612  0.30215 ...       0   0.000000   
51791  [-1.96499512e-01 -2.11201787e-01 -3.53611588e-...       0   0.375000   
51792  [ 0.02891248  0.014528   -0.07090136 -0.216146...       0   0.500000   
51793  [-2.61002518e-02 -4.08321083e-01  4.13673431e-...       0  -0.900000   
51794  [-0.7942242  -0.35421196 -0.1886899   0.171653...      -1   0.092593   

       subjectivity date_strings day_of_week  
0          0.900000   2020-12-31    Thursday  
1          0.538690   2020-12-31    Thursday  
2          0.500000   2020-12-31    Thursday  
3          0.525000   2020-12-31    Thursday  
4          0.000000   2020-12-31    Thursday  
...             ...          ...         ...  
51790      0.000000    2021-2-16     Tuesday  
51791      0.500000    2021-2-16     Tuesday  
51792      0.500000    2021-2-16     Tuesday  
51793      1.000000    2021-2-16     Tuesday  
51794      0.410185    2021-2-16     Tuesday  

[51795 rows x 11 columns]

In [97]:
len(df.groupby('date_strings')['date_strings'].count())

43

In [11]:
stocks = ['GME.csv','BB.csv','AMC.csv','NOK.csv','PLTR.csv','TSLA.csv']
df_stocks = []
for stock in stocks:
    df_stocks.append(pd.read_csv(stock))
    
filtered_stocks = []
for frame in df_stocks:
    frame.Date = pd.DatetimeIndex(data=frame.Date)
    filtered_stocks.append(frame[frame.Date >= datetime(2020, 12, 25)])
    
df_stocks = []
for frame in filtered_stocks:
    frame.Date = pd.DatetimeIndex(data=frame.Date,tz='Etc/Universal')
    frame.Date = frame.Date.apply(lambda timestamp : timestamp.tz_convert('US/Eastern'))
    df_stocks.append(frame)
    
for frame in df_stocks:
    frame['up_today'] = frame.apply(is_stock_up,axis=1)

In [12]:
time_erase = df_stocks[0].iloc[3,0]
def rotate_df_series(movements):
    first_half = [movements[-1]]
    second_half = movements[:-1]
    first_half.extend(second_half)
    return first_half

def remove_first_row(df):
    row = df[df.Date<=time_erase].index
    return df.drop(row)

In [13]:
cleanned_frames = []
for frame in df_stocks:
    movements = list(frame.up_today)
    prev_day = rotate_df_series(movements)
    frame['up_prev_day'] = prev_day
    frame = remove_first_row(frame)
    cleanned_frames.append(frame)
df_stocks = []
for frame in cleanned_frames:
    frame = remove_first_row(frame)
    df_stocks.append(frame)

In [14]:
for frame in df_stocks:
    dates = get_date(frame.Date)
    frame['date_strings'] = dates
    frame['day_of_week'] = get_day_of_week(frame.Date,shift_days=True)
    

In [15]:
for frame in df_stocks:
    frame['pct_change'] = -(frame['Open'] - frame['Close']) / frame['Open']
df_stocks[0].head()

Date       Open       High    Low      Close  \
4755 2021-01-03 19:00:00-05:00  19.000000  19.100000  17.15  17.250000   
4756 2021-01-04 19:00:00-05:00  17.350000  18.080000  17.23  17.370001   
4757 2021-01-05 19:00:00-05:00  17.340000  18.980000  17.33  18.360001   
4758 2021-01-06 19:00:00-05:00  18.469999  19.450001  18.02  18.080000   
4759 2021-01-07 19:00:00-05:00  18.180000  18.299999  17.08  17.690001   

      Adj Close    Volume  up_today  up_prev_day date_strings day_of_week  \
4755  17.250000  10022500     False        False     2021-1-4      Monday   
4756  17.370001   4961500      True        False     2021-1-5     Tuesday   
4757  18.360001   6056200      True         True     2021-1-6   Wednesday   
4758  18.080000   6129300     False         True     2021-1-7    Thursday   
4759  17.690001   6482000     False        False     2021-1-8      Friday   

      pct_change  
4755   -0.092105  
4756    0.001153  
4757    0.058824  
4758   -0.021115  
4759   -0.026953

In [20]:
def clean_open(open_price,close_price):
    open_price = list(open_price)
    close_price = list(close_price)
    cleanned_prices = [open_price[0]]
    cleanned_prices.extend(close_price[:-1])
    return cleanned_prices

In [22]:
for frame in df_stocks:
    frame['Open'] = clean_open(frame.Open,frame.Close)
df_stocks[0].head()

Date       Open       High    Low      Close  \
4755 2021-01-03 19:00:00-05:00  19.000000  19.100000  17.15  17.250000   
4756 2021-01-04 19:00:00-05:00  17.250000  18.080000  17.23  17.370001   
4757 2021-01-05 19:00:00-05:00  17.370001  18.980000  17.33  18.360001   
4758 2021-01-06 19:00:00-05:00  18.360001  19.450001  18.02  18.080000   
4759 2021-01-07 19:00:00-05:00  18.080000  18.299999  17.08  17.690001   

      Adj Close    Volume  up_today  up_prev_day date_strings day_of_week  \
4755  17.250000  10022500     False        False     2021-1-4      Monday   
4756  17.370001   4961500      True        False     2021-1-5     Tuesday   
4757  18.360001   6056200      True         True     2021-1-6   Wednesday   
4758  18.080000   6129300     False         True     2021-1-7    Thursday   
4759  17.690001   6482000     False        False     2021-1-8      Friday   

      pct_change  
4755   -0.092105  
4756    0.001153  
4757    0.058824  
4758   -0.021115  
4759   -0.026953

In [23]:
for frame in df_stocks:
    frame['pct_change'] = -(frame['Open'] - frame['Close']) / frame['Open']
df_stocks[0].head()

Date       Open       High    Low      Close  \
4755 2021-01-03 19:00:00-05:00  19.000000  19.100000  17.15  17.250000   
4756 2021-01-04 19:00:00-05:00  17.250000  18.080000  17.23  17.370001   
4757 2021-01-05 19:00:00-05:00  17.370001  18.980000  17.33  18.360001   
4758 2021-01-06 19:00:00-05:00  18.360001  19.450001  18.02  18.080000   
4759 2021-01-07 19:00:00-05:00  18.080000  18.299999  17.08  17.690001   

      Adj Close    Volume  up_today  up_prev_day date_strings day_of_week  \
4755  17.250000  10022500     False        False     2021-1-4      Monday   
4756  17.370001   4961500      True        False     2021-1-5     Tuesday   
4757  18.360001   6056200      True         True     2021-1-6   Wednesday   
4758  18.080000   6129300     False         True     2021-1-7    Thursday   
4759  17.690001   6482000     False        False     2021-1-8      Friday   

      pct_change  
4755   -0.092105  
4756    0.006957  
4757    0.056995  
4758   -0.015251  
4759   -0.021571

In [28]:
df_stocks[0].groupby('day_of_week')['pct_change'].mean()

day_of_week
Friday       0.103654
Monday       0.038168
Thursday     0.008046
Tuesday      0.027943
Wednesday    0.231582
Name: pct_change, dtype: float64

In [60]:
sp500 = pd.read_csv('SP500.csv')
sp500

Date      Open      High       Low    Close* Adj Close**  \
0     31-Mar-21  3,967.25  3,994.41  3,966.98  3,972.89    3,972.89   
1     30-Mar-21  3,963.34  3,968.01  3,944.35  3,958.55    3,958.55   
2     29-Mar-21  3,969.31  3,981.83  3,943.25  3,971.09    3,971.09   
3     26-Mar-21  3,917.12  3,978.19  3,917.12  3,974.54    3,974.54   
4     25-Mar-21  3,879.34  3,919.54  3,853.50  3,909.52    3,909.52   
...         ...       ...       ...       ...       ...         ...   
3096  10-Dec-08    892.17    908.27    885.45    899.24      899.24   
3097   9-Dec-08    906.48    916.26    885.38    888.67      888.67   
3098   8-Dec-08    882.71    918.57    882.71     909.7       909.7   
3099   5-Dec-08    844.43    879.42    818.41    876.07      876.07   
3100        NaN       NaN       NaN       NaN       NaN         NaN   

             Volume  
0     2,416,599,388  
1     4,103,570,000  
2     4,619,840,000  
3     5,467,850,000  
4     4,940,800,000  
...             ...  
3096  5,942,130,000  
3097  5,693,110,000  
3098  6,553,600,000  
3099  6,165,370,000  
3100            NaN  

[3101 rows x 7 columns]

In [61]:
sp500['Date'] = pd.DatetimeIndex(data=sp500.Date)
sp500 = sp500.dropna()

In [62]:
sp500['day_of_week'] = get_day_of_week(sp500.Date)
sp500

<ipython-input-62-91fcc00a339b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500['day_of_week'] = get_day_of_week(sp500.Date)


Date      Open      High       Low    Close* Adj Close**  \
0    2021-03-31  3,967.25  3,994.41  3,966.98  3,972.89    3,972.89   
1    2021-03-30  3,963.34  3,968.01  3,944.35  3,958.55    3,958.55   
2    2021-03-29  3,969.31  3,981.83  3,943.25  3,971.09    3,971.09   
3    2021-03-26  3,917.12  3,978.19  3,917.12  3,974.54    3,974.54   
4    2021-03-25  3,879.34  3,919.54  3,853.50  3,909.52    3,909.52   
...         ...       ...       ...       ...       ...         ...   
3095 2008-12-11    898.35    904.63    868.73    873.59      873.59   
3096 2008-12-10    892.17    908.27    885.45    899.24      899.24   
3097 2008-12-09    906.48    916.26    885.38    888.67      888.67   
3098 2008-12-08    882.71    918.57    882.71     909.7       909.7   
3099 2008-12-05    844.43    879.42    818.41    876.07      876.07   

             Volume day_of_week  
0     2,416,599,388   Wednesday  
1     4,103,570,000     Tuesday  
2     4,619,840,000      Monday  
3     5,467,850,000      Friday  
4     4,940,800,000    Thursday  
...             ...         ...  
3095  5,513,840,000    Thursday  
3096  5,942,130,000   Wednesday  
3097  5,693,110,000     Tuesday  
3098  6,553,600,000      Monday  
3099  6,165,370,000      Friday  

[3100 rows x 8 columns]

In [63]:
mapper = {'Close*':'Close',
         'Adj Close**': 'Adj Close'}

sp500 = sp500.rename(mapper=mapper,axis=1)
sp500

Date      Open      High       Low     Close Adj Close  \
0    2021-03-31  3,967.25  3,994.41  3,966.98  3,972.89  3,972.89   
1    2021-03-30  3,963.34  3,968.01  3,944.35  3,958.55  3,958.55   
2    2021-03-29  3,969.31  3,981.83  3,943.25  3,971.09  3,971.09   
3    2021-03-26  3,917.12  3,978.19  3,917.12  3,974.54  3,974.54   
4    2021-03-25  3,879.34  3,919.54  3,853.50  3,909.52  3,909.52   
...         ...       ...       ...       ...       ...       ...   
3095 2008-12-11    898.35    904.63    868.73    873.59    873.59   
3096 2008-12-10    892.17    908.27    885.45    899.24    899.24   
3097 2008-12-09    906.48    916.26    885.38    888.67    888.67   
3098 2008-12-08    882.71    918.57    882.71     909.7     909.7   
3099 2008-12-05    844.43    879.42    818.41    876.07    876.07   

             Volume day_of_week  
0     2,416,599,388   Wednesday  
1     4,103,570,000     Tuesday  
2     4,619,840,000      Monday  
3     5,467,850,000      Friday  
4     4,940,800,000    Thursday  
...             ...         ...  
3095  5,513,840,000    Thursday  
3096  5,942,130,000   Wednesday  
3097  5,693,110,000     Tuesday  
3098  6,553,600,000      Monday  
3099  6,165,370,000      Friday  

[3100 rows x 8 columns]

In [65]:
for col in sp500.columns[1:7]:
    sp500[col] = sp500[col].str.replace(',','')
    sp500[col] = pd.to_numeric(sp500[col],errors='raise')
sp500.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3100 entries, 0 to 3099
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         3100 non-null   datetime64[ns]
 1   Open         3100 non-null   float64       
 2   High         3100 non-null   float64       
 3   Low          3100 non-null   float64       
 4   Close        3100 non-null   float64       
 5   Adj Close    3100 non-null   float64       
 6   Volume       3100 non-null   int64         
 7   day_of_week  3100 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 218.0+ KB


In [68]:
sp500['Open'] = clean_open(sp500.Open,sp500.Close)

In [69]:
sp500['pct_change'] = -(sp500['Open'] - sp500['Close']) / sp500['Open']

In [80]:
sp500['up_today'] = sp500['Open'] > sp500['Close']
sp500

Date     Open     High      Low    Close  Adj Close      Volume  \
0    2021-03-31  3967.25  3994.41  3966.98  3972.89    3972.89  2416599388   
1    2021-03-30  3972.89  3968.01  3944.35  3958.55    3958.55  4103570000   
2    2021-03-29  3958.55  3981.83  3943.25  3971.09    3971.09  4619840000   
3    2021-03-26  3971.09  3978.19  3917.12  3974.54    3974.54  5467850000   
4    2021-03-25  3974.54  3919.54  3853.50  3909.52    3909.52  4940800000   
...         ...      ...      ...      ...      ...        ...         ...   
3095 2008-12-11   879.73   904.63   868.73   873.59     873.59  5513840000   
3096 2008-12-10   873.59   908.27   885.45   899.24     899.24  5942130000   
3097 2008-12-09   899.24   916.26   885.38   888.67     888.67  5693110000   
3098 2008-12-08   888.67   918.57   882.71   909.70     909.70  6553600000   
3099 2008-12-05   909.70   879.42   818.41   876.07     876.07  6165370000   

     day_of_week  pct_change  up_today  
0      Wednesday    0.001422     False  
1        Tuesday   -0.003609      True  
2         Monday    0.003168     False  
3         Friday    0.000869     False  
4       Thursday   -0.016359      True  
...          ...         ...       ...  
3095    Thursday   -0.006979      True  
3096   Wednesday    0.029362     False  
3097     Tuesday   -0.011754      True  
3098      Monday    0.023665     False  
3099      Friday   -0.036968      True  

[3100 rows x 10 columns]

In [81]:
sp500.groupby(['day_of_week','up_today']).count()

Date  Open  High  Low  Close  Adj Close  Volume  \
day_of_week up_today                                                    
Friday      False      288   288   288  288    288        288     288   
            True       332   332   332  332    332        332     332   
Monday      False      264   264   264  264    264        264     264   
            True       319   319   319  319    319        319     319   
Thursday    False      272   272   272  272    272        272     272   
            True       352   352   352  352    352        352     352   
Tuesday     False      283   283   283  283    283        283     283   
            True       354   354   354  354    354        354     354   
Wednesday   False      286   286   286  286    286        286     286   
            True       350   350   350  350    350        350     350   

                      pct_change  
day_of_week up_today              
Friday      False            288  
            True             332  
Monday      False            264  
            True             319  
Thursday    False            272  
            True             352  
Tuesday     False            283  
            True             354  
Wednesday   False            286  
            True             350

In [82]:
sp500.to_csv('SP500.csv')